In [1]:
#!pip install --upgrade transformers==2.9.0

In [2]:
#!pip install --upgrade pytorch_lightning==0.7.5

In [3]:
#!pip install sentencepiece

In [ ]:
#!pip install t5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 489.9/489.9 MB 39.0 MB/s eta 0:00:01

In [5]:
#!pip install pyterrier

In [6]:
#!pip install python-terrier

# Import Libraries

In [7]:
import numpy as np
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

ModuleNotFoundError: ignored

## Connect To Google Drive And Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive/MyDrive/project_data'

In [ ]:
!ls '/content/drive/MyDrive/project_data/scifact'

In [ ]:
data_directory = '/content/drive/MyDrive/project_data'

# Load Meta-Data

In [ ]:
corpus = pd.read_json(data_directory + '/scifact/corpus.jsonl', lines=True, dtype=str)

corpus

# Load Test data and qrels from .csv files

In [ ]:
#test data
df_test = pd.read_csv(data_directory + '/scifact/test.csv', sep='\t', dtype=str)
print(df_test.shape)
print(df_test['query'].apply(len).mean())
print(df_test['text'].apply(len).mean())
df_test

In [ ]:
#test_data
df_test = pd.read_csv(data_directory + '/scifact/test.csv', sep='\t', dtype=str)
df_test2 = df_test[['qid', 'query']]
df_test2.to_csv('my_test_queries.csv', sep = '\t', index=False, header=False)
test_query = pt.io.read_topics('my_test_queries.csv', format='singleline')
test_query

In [ ]:
#test qrels
df_test_qrels = pd.read_csv(data_directory + '/scifact/qrels/test.tsv', sep='\t', dtype=str)

df_test_qrels

In [ ]:
#source data
test_source = test_query['query']
test_source

#target data
test_target = df_test['text']

test_target

# T5 - Query Rephrasing

In [ ]:
#INSPIRED BY!
#https://github.com/ramsrigouthamg/Paraphrase-any-question-with-T5-Text-To-Text-Transfer-Transformer-
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(15)


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

In [ ]:
all_outputs = []
for i in range(len(test_source)):
  text =  "paraphrase: " + test_source.iloc[i] + " </s>"


  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.98,
      early_stopping=True,
      num_return_sequences=5
  )

  final_outputs =[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != test_source.iloc[i].lower() and sent not in final_outputs:
          final_outputs.append(sent)
  all_outputs.append(final_outputs)

  if i == 0:
    print ("\nOriginal Question ::")
    print (test_source.iloc[i])
    print ("\n")
    print ("Paraphrased Questions :: ")
    for i, final_output in enumerate(final_outputs):
        print("{}: {}".format(i, final_output))

  if i % 10 == 0:
    print(i,  " out of ", len(test_source))

In [ ]:
rewritten_text1 = []
rewritten_text2 = []
rewritten_text3 = []
rewritten_text4 = []
rewritten_text5 = []

counter = 0

for i in range(len(all_outputs)):
  rewritten_text1.append(all_outputs[i][0])

for i in range(len(all_outputs)):
  if len(all_outputs[i]) < 2:
    counter += 1
    rewritten_text2.append(all_outputs[i][0])
  else:
    rewritten_text2.append(all_outputs[i][1])

for i in range(len(all_outputs)):
  if len(all_outputs[i]) < 3:
    rewritten_text3.append(all_outputs[i][0])
  else:
    rewritten_text3.append(all_outputs[i][2])

for i in range(len(all_outputs)):
  if len(all_outputs[i]) < 4:
    rewritten_text4.append(all_outputs[i][0])
  else:
    rewritten_text4.append(all_outputs[i][3])

for i in range(len(all_outputs)):
  if len(all_outputs[i]) < 5:
    rewritten_text5.append(all_outputs[i][0])
  else:
    rewritten_text5.append(all_outputs[i][4])

print(counter)

# Evaluation using BM25

In [ ]:
corpus = corpus.rename(columns={'_id': 'docno'})
corpus

In [ ]:
indexer = pt.DFIndexer("./indexes_scifact/both", overwrite=True, blocks=True,verbose=True, stemmer='porter', stopwords='terrier', tokenizer = 'english')
index_ref = indexer.index(corpus["text"], corpus["docno"])
index = pt.IndexFactory.of(index_ref)

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [ ]:
new_queries1 = pd.DataFrame()
new_queries1['qid'] = test_query['qid']
new_queries1['query'] = test_source

new_queries1 = pd.DataFrame()
new_queries1['qid'] = test_query['qid']
new_queries1['query'] = rewritten_text1

new_queries2 = pd.DataFrame()
new_queries2['qid'] = test_query['qid']
new_queries2['query'] = rewritten_text2

new_queries3 = pd.DataFrame()
new_queries3['qid'] = test_query['qid']
new_queries3['query'] = rewritten_text3

new_queries4 = pd.DataFrame()
new_queries4['qid'] = test_query['qid']
new_queries4['query'] = rewritten_text4

new_queries5 = pd.DataFrame()
new_queries5['qid'] = test_query['qid']
new_queries5['query'] = rewritten_text5

In [ ]:
new_queries1['query'] = new_queries1['query'].str.replace('%','')
new_queries1['query'] = new_queries1['query'].str.replace('?','')
new_queries1['query'] = new_queries1['query'].str.replace('\'','')
new_queries1['query'] = new_queries1['query'].str.replace('(','')
new_queries1['query'] = new_queries1['query'].str.replace(')','')
new_queries1['query'] = new_queries1['query'].str.replace(':','')
new_queries1['query'] = new_queries1['query'].str.replace('/','')

new_queries2['query'] = new_queries2['query'].str.replace('%','')
new_queries2['query'] = new_queries2['query'].str.replace('?','')
new_queries2['query'] = new_queries2['query'].str.replace('\'','')
new_queries2['query'] = new_queries2['query'].str.replace('(','')
new_queries2['query'] = new_queries2['query'].str.replace(')','')
new_queries2['query'] = new_queries2['query'].str.replace(':','')
new_queries2['query'] = new_queries2['query'].str.replace('/','')

new_queries3['query'] = new_queries3['query'].str.replace('%','')
new_queries3['query'] = new_queries3['query'].str.replace('?','')
new_queries3['query'] = new_queries3['query'].str.replace('\'','')
new_queries3['query'] = new_queries3['query'].str.replace('(','')
new_queries3['query'] = new_queries3['query'].str.replace(')','')
new_queries3['query'] = new_queries3['query'].str.replace(':','')
new_queries3['query'] = new_queries3['query'].str.replace('/','')

new_queries4['query'] = new_queries4['query'].str.replace('%','')
new_queries4['query'] = new_queries4['query'].str.replace('?','')
new_queries4['query'] = new_queries4['query'].str.replace('\'','')
new_queries4['query'] = new_queries4['query'].str.replace('(','')
new_queries4['query'] = new_queries4['query'].str.replace(')','')
new_queries4['query'] = new_queries4['query'].str.replace(':','')
new_queries4['query'] = new_queries4['query'].str.replace('/','')

new_queries5['query'] = new_queries5['query'].str.replace('%','')
new_queries5['query'] = new_queries5['query'].str.replace('?','')
new_queries5['query'] = new_queries5['query'].str.replace('\'','')
new_queries5['query'] = new_queries5['query'].str.replace('(','')
new_queries5['query'] = new_queries5['query'].str.replace(')','')
new_queries5['query'] = new_queries5['query'].str.replace(':','')
new_queries5['query'] = new_queries5['query'].str.replace('/','')

In [ ]:
comb_rewritten_texts = []
for i in range(len(rewritten_text1)):
  res = test_source + " " + rewritten_text1[i] + " " + rewritten_text2[i] + " " + rewritten_text3[i] + " " + rewritten_text4[i] + " " + rewritten_text5[i]
  comb_rewritten_texts.append(res)

new_queries6 = pd.DataFrame()
new_queries6['qid'] = test_query['qid']
new_queries6['query'] = comb_rewritten_texts

In [ ]:
test_qrels = df_test_qrels.rename(columns={"query-id": "qid", "corpus-id" : "docno", "score": "label"})
test_qrels['iteration'] = 0
qrels = test_qrels.astype({'label': 'int32'})
qrels

In [ ]:
final_res_ndcg = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries1,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries1,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries1,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries1,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg)
print(final_res_map)
print(final_res_precision)
print(final_res_recall)

In [ ]:
final_res_ndcg2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg2)
print(final_res_map2)
print(final_res_precision2)
print(final_res_recall2)

In [ ]:
final_res_ndcg3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg3)
print(final_res_map3)
print(final_res_precision3)
print(final_res_recall3)

In [ ]:
final_res_ndcg4 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries4,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map4 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries4,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision4 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries4,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall4 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries4,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg4)
print(final_res_map4)
print(final_res_precision4)
print(final_res_recall4)

In [ ]:
final_res_ndcg5 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries5,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map5 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries5,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision5 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries5,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall5 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries5,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg5)
print(final_res_map5)
print(final_res_precision5)
print(final_res_recall5)

In [ ]:
final_res_ndcg6 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries6,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map6 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries6,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision6 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries6,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall6 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries6,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg6)
print(final_res_map6)
print(final_res_precision6)
print(final_res_recall6)